In [1]:
from transformers import AdamW, AutoModelForSeq2SeqLM,AutoTokenizer
import os
import pandas as pd

In [2]:

import torch
import os
import pandas as pd
import numpy as np
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm_notebook
import sacrebleu

In [3]:
model_directory = "/model/transformers_files/"
cache_dir_models = model_directory + "default_models/"
cache_dir_tokenizers = model_directory + "tokenizers/"
model_name = "google/mt5-small"

In [4]:
# Скачивание модели и токенизатора
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, cache_dir=cache_dir_models)
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir_tokenizers, legacy=False)

C:\Users\Admin\.virtualenvs\Translave\lib\site-packages\transformers\convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [5]:
folder = '/model'
model_folder='/model/modelmT5'
tokenizer_folder = '/model/tokenizermT5'

In [6]:
os.getcwd()

'C:\\Users\\Admin\\PycharmProjects\\Translave\\model'

In [7]:
# Сохраняем токенизатор
tokenizer.save_pretrained(tokenizer_folder)
# Сохраняем модель
model.save_pretrained(model_folder)

In [8]:
# Создаем папки, если их еще нет
tokenizer_folder = os.path.join('path', 'Tokenizer')
model_folder = os.path.join('path', 'Model')

In [9]:
model.config.max_length=40

In [10]:
len(tokenizer.vocab)

250100

In [11]:
train_df = pd.read_csv("C:\\Users\\Admin\\PycharmProjects\\Translave\\model\\train_data.csv").astype(str)
eval_df = pd.read_csv("C:\\Users\\Admin\\PycharmProjects\\Translave\\model\\validation_data.csv").astype(str)

In [12]:
train_df.sample(5)


,prefix,input_text,target_text
105753,rus.eng,<rus.eng> Том хотел пригласить Мэри на свидани...,"Tom wanted to ask Mary to go out with him, but..."
214660,rus.eng,<rus.eng> Японки часто выглядят маленькими и и...,Japanese women tend to look tiny and delicate.
312155,rus.eng,<rus.eng> Я помог Тому передвинуть стол.,I helped Tom move his desk.
491987,eng.rus,<eng.rus> I don't think that I'd change anythi...,"Не думаю, что я бы что-то изменил."
17202,rus.eng,"<rus.eng> Ты не мог бы оставить нас одних, пож...","Would you leave us alone, please?"


In [13]:
train_df['prefix'].unique()

array(['rus.eng', 'eng.rus'], dtype=object)

In [14]:
LANG_TOKEN_MAPPING = {
    '<rus.eng>': '',
    '<eng.rus>': ''
}

In [15]:
special_tokens_dict = {'additional_special_tokens': list(LANG_TOKEN_MAPPING)}
special_tokens_dict

{'additional_special_tokens': ['<rus.eng>', '<eng.rus>']}

In [16]:
tokenizer.add_special_tokens(special_tokens_dict)

2

In [17]:
tokenizer.all_special_tokens

['</s>', '<unk>', '<pad>', '<eng.rus>', '<rus.eng>']

In [18]:
model.config.vocab_size

250112

In [19]:
model.resize_token_embeddings(len(tokenizer))

Embedding(250102, 512)

In [20]:
def encode_str(text, tokenizer, seq_len):
    # Tokenize,pad to max length and encode to ids
    # Returns tensor with tocken ids"""
    input_ids = tokenizer.encode(
      text=text,
      return_tensors = 'pt',
      padding = 'max_length',
      truncation = True,
      max_length = seq_len)

    return input_ids[0]


In [21]:
max_seq_len = 40

In [22]:
t1=encode_str(' This is a test',tokenizer, max_seq_len)
print(t1)
tokens = tokenizer.convert_ids_to_tokens(t1)
print(tokens)

tensor([1494,  339,  259,  262, 2978,    1,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0])
['▁This', '▁is', '▁', 'a', '▁test', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


In [23]:
t2=encode_str(' тест для конвертации',tokenizer,max_seq_len)
print(t2)
tokens=tokenizer.convert_ids_to_tokens(t2)
print(tokens)

tensor([ 17019,    259,    735, 132896,   7547,      1,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0])
['▁тест', '▁', 'для', '▁конверт', 'ации', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
